In [18]:
import logging
import xgboost as xgb
from sklearn.model_selection import train_test_split
# https://medium.com/the-innovation/multi-label-classification-example-with-multioutputclassifier-and-xgboost-in-python-98c84c7d379f
#from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from dataset import create_dataset
#from ml_model import train_model

from utils import read_yaml


In [4]:

logging.basicConfig()
logging.root.setLevel(logging.INFO)
log = logging.getLogger('Hack4NF')


log.info('Reading config files')
# Read configuration files
config_path = "config/resources.yaml"
config = read_yaml(config_path)
genie = config["genie"]


# Read synapses credentials
secrets_path = ".secrets/synapses_credentials.yaml"
credentials = read_yaml(secrets_path)

INFO:Hack4NF:Reading config files


In [5]:
log.info('Creating dataset')
dataset = create_dataset(genie)

INFO:Hack4NF:Creating dataset
INFO:dataset:Reading dataset from: data//dataset.csv
C:\Users\lucas\Projects\hack4nf\Hack4NF\Hack4NF-2022\dataset.py:46: DtypeWarning: Columns (4,5,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(dataset_file, sep=";")


In [6]:
len(dataset)

146565

In [7]:
dataset.head()

,Unnamed: 0,PATIENT_ID,SEX,PRIMARY_RACE,SECONDARY_RACE,TERTIARY_RACE,ETHNICITY,BIRTH_YEAR,CENTER,INT_CONTACT,...,ZNRF3_DNP,ZNRF3_INS,ZNRF3_ONP,ZNRF3_SNP,ZRSR2_DEL,ZRSR2_DNP,ZRSR2_INS,ZRSR2_SNP,ZSWIM4_DEL,ZSWIM4_SNP
0,0,GENIE-VICC-101416,Female,White,Not collected,Not collected,Non-Spanish/non-Hispanic,1961.0,VICC,19225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,GENIE-VICC-102225,Female,White,Not collected,Not collected,Non-Spanish/non-Hispanic,1982.0,VICC,12057,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,GENIE-VICC-102424,Female,White,Not collected,Not collected,Non-Spanish/non-Hispanic,1952.0,VICC,23505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,GENIE-VICC-102966,Male,White,Not collected,Not collected,Non-Spanish/non-Hispanic,1951.0,VICC,23426,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,GENIE-VICC-103244,Female,Unknown,Not collected,Not collected,Unknown,1964.0,VICC,18267,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
dataset[genie["targets"]].isna().sum()

NF1_DEL    15272
NF1_DNP    15272
NF1_INS    15272
NF1_ONP    15272
NF1_SNP    15272
NF2_DEL    15272
NF2_DNP    15272
NF2_INS    15272
NF2_ONP    15272
NF2_SNP    15272
dtype: int64

In [9]:
dataset[dataset[genie["targets"]].isna()].head()

,Unnamed: 0,PATIENT_ID,SEX,PRIMARY_RACE,SECONDARY_RACE,TERTIARY_RACE,ETHNICITY,BIRTH_YEAR,CENTER,INT_CONTACT,...,ZNRF3_DNP,ZNRF3_INS,ZNRF3_ONP,ZNRF3_SNP,ZRSR2_DEL,ZRSR2_DNP,ZRSR2_INS,ZRSR2_SNP,ZSWIM4_DEL,ZSWIM4_SNP
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
ds = dataset.dropna(subset=genie["targets"])

In [11]:
len(ds)

131293

In [12]:
with open('data/genie_mutations_features.txt') as f:
    mutation_cols = f.read().splitlines()
features = set(mutation_cols) - set(genie["targets"])
features = list(features)

In [13]:
len(mutation_cols)

5438

In [14]:
len(features)

5428

In [15]:
X_train, X_test, y_train, y_test = train_test_split(ds[features].astype(int).values, ds[genie["targets"]].astype(int).values,
                                                    test_size=0.20, random_state=42)


In [20]:
help(xgb)

Help on package xgboost:

NAME
    xgboost

PACKAGE CONTENTS
    _typing

FILE
    (built-in)




In [19]:
classifier = MultiOutputClassifier(xgb.XGBClassifier(learning_rate=0.05, 
                                                 n_estimators=5000, 
                                                 subsample=0.7))

clf = Pipeline([('classify', classifier)])

print (clf)



AttributeError: module 'xgboost' has no attribute 'XGBClassifier'

In [ ]:
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
num_round = 100

In [ ]:
clf = xgb.XGBClassifier(tree_method="hist")


In [ ]:
y_test.shape

In [ ]:
y.shape

In [ ]:
# from sklearn.ensemble import RandomForestClassifier


# clf = RandomForestClassifier(max_depth=16, random_state=0)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_test)
# predictions.sum(axis=0)

In [ ]:
a

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(learning_rate=0.05, 
                                                 n_estimators=5000, 
                                                 subsample=0.7))

clf = Pipeline([('classify', classifier)])

print (clf)



In [ ]:
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

In [ ]:
yhat = clf.predict(X_test)

In [ ]:
auc_y1 = roc_auc_score(y_test[:,0],yhat[:,0])
auc_y2 = roc_auc_score(y_test[:,1],yhat[:,1])
auc_y3 = roc_auc_score(y_test[:,2],yhat[:,2])
auc_y4 = roc_auc_score(y_test[:,3],yhat[:,3])
auc_y5 = roc_auc_score(y_test[:,4],yhat[:,4])

print("ROC AUC y1: %.4f, y2: %.4f, y3: %.4f, y4: %.4f, y5: %.4f" % (auc_y1, auc_y2, auc_y3, auc_y4, auc_y5))


In [ ]:
cm_y1 = confusion_matrix(y_test[:,0],yhat[:,0])
cm_y2 = confusion_matrix(y_test[:,1],yhat[:,1])
cm_y3 = confusion_matrix(y_test[:,2],yhat[:,2])
cm_y4 = confusion_matrix(y_test[:,3],yhat[:,3])
cm_y5 = confusion_matrix(y_test[:,4],yhat[:,4])

print (cm_y1)


In [ ]:
cr_y1 = classification_report(y_test[:,0],yhat[:,0])
cr_y2 = classification_report(y_test[:,1],yhat[:,1])
cr_y3 = classification_report(y_test[:,2],yhat[:,2])
cr_y4 = classification_report(y_test[:,3],yhat[:,3])
cr_y5 = classification_report(y_test[:,4],yhat[:,4])

print (cr_y1)

In [ ]:
# clf.fit(X_train, y_train,
#         eval_set=[(X_train, y_train), (X_test, y_test)],
#         eval_metric='mlogloss',
#         verbose=True)

In [ ]:
from sklearn.datasets import make_multilabel_classification
import numpy as np

X, y = make_multilabel_classification(
    n_samples=32, n_classes=5, n_labels=3, random_state=0
)
clf = xgb.XGBClassifier(tree_method="hist")
clf.fit(X, y)
np.testing.assert_allclose(clf.predict(X), y)

In [ ]:
y

In [ ]:
# model_path = f"model_{target}"
# create_folder(model_path)
ds = ds.fillna(0)


X_train, X_test, y_train, y_test = train_test_split(ds[features], ds[target],
                                                    test_size=0.20, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params_k = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'subsample': 0.8,
    'subsample_freq': 1,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'max_bin': 300,
    'n_estimators': 2000,
    'boost_from_average': False,
    "random_seed": 42}

model_gbm = lgb.train(params_k, train_data, valid_sets=[test_data],
                      num_boost_round=5000, early_stopping_rounds=25,
                      verbose_eval=50)
joblib.dump(model_gbm, f'{model_path}/model_lgb.pkl')

# evaluate_model(model_gbm, X_test, y_test, model_path)

# model_interpretability(model_gbm, X_test, model_path)